# Split loaded documents using trained splitter model

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

In [2]:
from datetime import datetime
import os

from IPython.display import display, Markdown
import torch

from models.load_utils import load_docs_from_jsonl, save_docs_to_jsonl
from models.split_model import ModelTextSplitter, MarkdownSyntacticEmbeddingSplitter
from models.split_utils import get_openai_embedder

In [3]:
# List all files in the new directory
split_path = "/home/public/iloveconference/split/model"
files_in_new = os.listdir(split_path)
files_in_new.sort()  # Sort files alphabetically
for file in files_in_new:
    print(file)

2023-04-11.pkl
2023-04-12.pkl
2023-09-24.pkl


In [4]:
# configure
input_path = '../data/split/conference/2025-09-16.jsonl'
split_model_path = '/home/public/iloveconference/split/model/2023-09-24.pkl'
output_dir = '../data/split/conference'
today = datetime.today().strftime('%Y-%m-%d')

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
torch.cuda.empty_cache()
print(torch.cuda.is_available())
print("cuda total", torch.cuda.get_device_properties(0).total_memory)
print("cuda reserved", torch.cuda.memory_reserved(0))
print("cuda allocated", torch.cuda.memory_allocated(0))

True
cuda total 8139374592
cuda reserved 0
cuda allocated 0


## Load documents

In [7]:
docs = load_docs_from_jsonl(input_path)
len(docs)

136

## Create splits

In [8]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 18.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
# deprecated
text_splitter = ModelTextSplitter(split_model_path, split_threshold=0.55, chunk_size=500, anchor="anchor")
splits = text_splitter.split_documents(docs, verbose=True)
len(splits)

/home/promise/conference/models/.venv/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/promise/conference/models/.venv/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/promise/conference/models/.venv/lib/python3.10/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Pipeline fr

  0%|          | 0/136 [00:00<?, ?it/s]

1146

In [11]:
# 7x faster, but slightly lesser quality
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
openai_embedder = get_openai_embedder(openai)

mse_splitter = MarkdownSyntacticEmbeddingSplitter(embedder=openai_embedder, split_threshold=0.80)
splits = mse_splitter.split_documents(docs, verbose=True)
len(splits)

  0%|          | 0/136 [00:00<?, ?it/s]

1087

In [12]:
%%time
mse_splitter = MarkdownSyntacticEmbeddingSplitter()
splits = mse_splitter.split_documents(docs, verbose=True)
len(splits)

  0%|          | 0/136 [00:00<?, ?it/s]

CPU times: user 7.39 s, sys: 304 ms, total: 7.7 s
Wall time: 3min 36s


1149

## Save splits

In [13]:
filename = os.path.join(output_dir, f"{today}.jsonl")
save_docs_to_jsonl(splits, filename)

In [14]:
print(len(docs), len(splits))

136 1149
